In [1]:
import numpy as np
import json
import os
import colorsys
import PIL

from PIL import Image
from collections import Counter


In [14]:
fish_mask = './RGBdata/mask/180529-0955136_r-16.png'

In [15]:
def getLableInfo():
    with open('./RGBdata/labelTemplate.json') as json_file:
        return json.load(json_file)

def getIntToLabelDict():
    high_level_features = {}
    low_level_features = {}

    labelInfo = getLableInfo()
    
    for high_level_feature in labelInfo:
        classes_with_high_level_feature = []

        for low_level_feature in labelInfo[high_level_feature]:
            if low_level_feature != 'color':
                high_level_features[labelInfo[high_level_feature][low_level_feature]] = high_level_feature
                low_level_features[labelInfo[high_level_feature][low_level_feature]] = low_level_feature
    
    return high_level_features, low_level_features

def getMaskClasses(path, high_level_filter, low_level_filter):

    image = Image.open(path)
    arr = np.unique(np.array(image))


    high_level = []
    low_level = []


    for clss in arr:
        high_level.append(high_level_filter[clss])
        low_level.append(low_level_filter[clss])

    return list(set(high_level)), list(set(low_level))
    
    
    

def feature_count():

    high_index, low_index = getIntToLabelDict()

    classCollection = []

    high_level = []
    low_level = []

    for image_path in os.listdir('./RGBdata/mask'):
        high_level_classes, low_level_classes = getMaskClasses(f'./RGBdata/mask/{image_path}', high_index, low_index)
        high_level.extend(high_level_classes)
        low_level.extend(low_level_classes)
    
    
    high_level_count = Counter(high_level)
    low_level_count = Counter(low_level)
    
    return dict(high_level_count), dict(low_level_count)


high, low = feature_count()

high, low




({'Nematodes': 249,
  'Black lining': 451,
  'Filet': 539,
  'Bones/cartilage': 334,
  'White lining (cut surface)': 447,
  'Other/infection/parasite': 184,
  'Belt': 539,
  'Blood': 466,
  'Level of bleeding (overall)': 244,
  'Bile spot': 83,
  'Silver membrane': 52,
  'Skin remains': 25,
  'Color of flesh': 1},
 {'Brownish': 75,
  'Filet': 539,
  'Grey': 208,
  'Dark-Coiled': 248,
  'Dark': 404,
  'Light': 448,
  'Belt': 539,
  'Ventral fin': 213,
  'Caudal fin (tail)': 112,
  'Anal fin': 205,
  'Bruises': 94,
  'Dorsal fin': 201,
  'Ribs': 19,
  'Kidney blood': 98,
  'Reddish': 61,
  'Blood clots': 214,
  'Blood remains': 400,
  'Blood': 74,
  'Strong color': 66,
  'Overlap\t': 18,
  'Veins': 8,
  'Slightly reddish': 202,
  'Overlap': 151,
  'Light\t': 55,
  'Orange': 55,
  'Grey\t': 43,
  'Swimmbladder': 23,
  'Head bone': 19,
  'Light color': 46,
  'Moderately reddish': 16,
  'Pectoral fin': 49,
  'Pearl stripes': 10,
  'Backbone': 22,
  'Swimbladder': 17,
  'Very reddish': 11,
 

In [19]:
def gen_rgb_set(N):
    N = len(np.unique(arr))

    hsv_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]
    rgb_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples)
    return list(rgb_tuples)

def get_mask_coloring(path):
    image = Image.open(path)
    arr = np.array(image)
    l, w = arr.shape
    
    rgb_image = np.zeros((l, w, 3))

    N = len(np.unique(arr))

    hsv_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]
    rgb_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples)



get_mask_coloring(fish_mask)



6


In [59]:
def get_json(json_path):
    with open(json_path) as json_file:
        return json.load(json_file)

def get_catagories(og_labels):
    catagories = set()

    for key in og_labels:
        for sub_key in og_labels[key]:
            if sub_key != 'color':
                catagories.add(sub_key)
        catagories.add(key)

    return catagories

def gen_rgb_set(N):

    hsv_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]
    rgb_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples)
    return list(rgb_tuples)

def catagories_to_colormap(catagories):
    colormap = {}
    
    colorset = gen_rgb_set(len(catagories))

    for i, item in enumerate(catagories):
        colormap[item] = colorset[i]
    
    return colormap

def reverse_og_labels(og_labels):
    reversed_index = {}

    for key in og_labels:
        for sub_key in og_labels[key]:
            if sub_key != 'color':
                reversed_index[og_labels[key][sub_key]] = sub_key
    return reversed_index

def draw_original_mask(path, reversed_index, colormap):
    
    full_path = './../datasets/marel/RGBdata/mask/'+path

    mask = np.array(Image.open(full_path))
    h, w = mask.shape

    colored_mask = np.zeros((h,w,3))

    for label in np.unique(mask):
        colored_mask[mask==label] = colormap[reversed_index[label]]
    
    return Image.fromarray((colored_mask*255).astype(np.uint8))

def draw_six_class_mask(path, reversed_index, colormap):
    full_path = './../datasets/marel/RGBdata/six_class_mask/'+path

    mask = np.array(Image.open(full_path))
    h, w = mask.shape

    colored_mask = np.zeros((h,w,3))

    for label in np.unique(mask):

        colored_mask[mask==label] = colormap[reversed_index[label]]
    
    return Image.fromarray((colored_mask*255).astype(np.uint8))

def get_reversed_six_index():
    reversed_index = {}
    
    index = get_json('./../datasets/marel/RGBdata/six_class_labels.json')
    
    for key in index:
        reversed_index[index[key]['label']] = key

    return reversed_index
    

def get_colormap(path):
    og_labels = get_json('./../datasets/marel/RGBdata/labelTemplate.json')

    reversed_og_labels = reverse_og_labels(og_labels)
    
    catagories = get_catagories(og_labels)

    catagory_colormap = catagories_to_colormap(catagories)

    mask = draw_original_mask(path, reversed_og_labels, catagory_colormap)

    reversed_six_class = get_reversed_six_index()

    six_class_mask = draw_six_class_mask(path, reversed_six_class, catagory_colormap)


    return mask, six_class_mask

def merge_and_save_three_images(path):
    list_im = ['./../datasets/marel/RGBdata/org/'+path, './../datasets/marel/RGBdata/colored/mask/'+path, './../datasets/marel/RGBdata/colored/six_class/'+path]
    imgs    = [ PIL.Image.open(i) for i in list_im ]
    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

    # save that beautiful picture
    imgs_comb = PIL.Image.fromarray( imgs_comb)
    imgs_comb.save( './../datasets/marel/RGBdata/colored/merged/'+path )    

def main():
    for path in os.listdir('./../datasets/marel/RGBdata/org/'):
        merge_and_save_three_images(path)


main()

In [3]:
for path in os.listdir('./../datasets/marel/RGBdata/six_class_mask'):
    print(np.unique(np.array(Image.open('./../datasets/marel/RGBdata/six_class_mask/'+path))))

[0 1 2 3]
[0 1 2 3 4]
[0 1 2 3 4]
[0 1 2 3 4 5]
[0 1 2 3 4]
[0 1 4]
[0 1 2 4]
[0 1 2 3 4 5]
[0 1 2 4]
[0 1 2 3 4]
[0 1 2 3 4]
[0 1 2 4]
[0 1 2 3 4]
[0 1 2 4]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4]
[0 1 2 3 4]
[0 1 5]
[0 1 2 4 5]
[0 1 2 4 5]
[0 1 2 4]
[0 1 2 4]
[0 1 2 4]
[0 1 2 4]
[0 1 2 4]
[0 1 4]
[0 1 4]
[0 1 4 5]
[0 1 2 3 4]
[0 1 2 3 4 5]
[0 1 2]
[0 1 2 3 4 5]
[0 1 3 4]
[0 1 2 4]
[0 1 3 4]
[0 1 2 4]
[0 1 2 4]
[0 1 2 4]
[0 1 2 4 5]
[0 1 2 3 4]
[0 1 2 4]
[0 1 2 4]
[0 1 2 4 5]
[0 1 2 4 5]
[0 1 2 4]
[0 1 4 5]
[0 1 2 4]
[0 1 2 4]
[0 1 2 3 4]
[0 1 2 4]
[0 1 2 3 4]
[0 1 4]
[0 1 4]
[0 1 2 3 4 5]
[0 1 2 4 5]
[0 1 2 3]
[0 1 2 4 5]
[0 1 2 3 4]
[0 1 2 5]
[0 1 2 3 4 5]
[0 1 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4]
[0 1 2 3 4 5]
[0 1 2 4]
[0 1 2 4]
[0 1 2 3]
[0 1 2 4 5]
[0 1 2 4]
[0 1 2 3 4]
[0 1 2 4 5]
[0 1 2 4]
[0 1 2 3 4 5]
[0 1 2 5]
[0 1 2 4 5]
[0 1 2 4 5]
[0 1 2 4]
[0 1 3 4]
[0 1 2 3 4 5]
[0 1 2 4]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4]
[0 1 2 3 4]
[0 1 2 3 4]
[0 1 2 3 4 5]
